## PIP INSTALL

In [ ]:
!pip install openai

In [ ]:
!pip install langchain

In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33


In [ ]:
!pip install -qU langchain-google-vertexai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 2.1 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade --quiet  langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.0 MB/s eta 0:00:00


In [ ]:
!pip install -qU langchain-anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 5.5 MB/s eta 0:00:00


In [ ]:
!pip install -qU langchain-nvidia-ai-endpoints

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.6 MB/s eta 0:00:00


In [ ]:
!pip install -qU langchain-cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00


In [ ]:
%pip install -qU langchain-deepseek

# ENV PREPARATION

In [ ]:
import openai
import langchain
import getpass
import os
import numpy as np
from getpass import getpass
from langchain_openai import ChatOpenAI
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_anthropic import ChatAnthropic
from langchain_openai import AzureChatOpenAI
from langchain_cohere import ChatCohere
from langchain_deepseek import ChatDeepSeek

from tqdm import tqdm

In [ ]:
os.environ["COHERE_API_KEY"] = '' ## your key here

from langchain_cohere import ChatCohere

model = ChatCohere(model="command-r-plus")

In [ ]:
os.environ["ANTHROPIC_API_KEY"] = ''

model = ChatAnthropic(model="claude-3-5-sonnet-20240620")


In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
os.environ["DEEPSEEK_API_KEY"] = ""

# DATASET

In [ ]:
import json
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
questions_dict = {}
answers_dict = {}

with open('/content/drive/MyDrive/Colab Notebooks/Multi-supervisor/Expriments/train.jsonl', 'r') as file:
    for i, line in enumerate(file):
        item = json.loads(line)

        # put questions and options in this dict
        questions_dict[f"question_{i+1}"] = {
            "question": item["question"],
            "options": item["options"]
        }

        # put answers in this dict
        answers_dict[f"answer_{i+1}"] = item["answer_idx"]

In [ ]:
numbers_question = 10

In [ ]:
# print first 10 records
def preview_dataset():
  print("Questions Dictionary:")
  for key in list(questions_dict.keys())[:numbers_question]:
      print(f"{key}: {questions_dict[key]}")

  print("\nAnswers Dictionary:")
  for key in list(answers_dict.keys())[:numbers_question]:
      print(f"{key}: {answers_dict[key]}")

preview_dataset()

Questions Dictionary:
question_1: {'question': 'A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?', 'options': {'A': 'Ampicillin', 'B': 'Ceftriaxone', 'C': 'Ciprofloxacin', 'D': 'Doxycycline', 'E': 'Nitrofurantoin'}}
question_2: {'question': 'A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions coul

# INITIALIZE MODEL

In [ ]:
# model = ChatOpenAI(model="gpt-3.5-turbo")
# model = ChatVertexAI(model="gemini-1.5-flash")
# model = ChatAnthropic(model="claude-3-5-sonnet-20240620")
# model = ChatCohere(model="command-r-plus-08-2024")
model = ChatDeepSeek(model="deepseek-chat")


In [ ]:
def messages(key):
    # convert into strings
    question_content = questions_dict[key]
    formatted_question = f"{question_content['question']}\n"
    for option_key, option_value in question_content['options'].items():
        formatted_question += f"{option_key}) {option_value}\n"

    message = [
        SystemMessage(content="Complete the following multiple-choice question and just return the letter represent the choice, return answer in format A)"),
        HumanMessage(content=formatted_question),
    ]
    return message

def get_answer():
  selected_options = []
  for key in list(questions_dict.keys())[:numbers_question]:
      message_chain = messages(key)
      output = model.invoke(message_chain)
      # print(output)
      print(output.usage_metadata)

      output_text = output.content
      for line in output_text.splitlines():
          if line:  # null line
              # print("---------------------")
              # print("selected")
              # print(line.split(')')[0])
              selected_options.append(line.split(')')[0])

      # print(selected_options)
  return selected_options

error_question = {}

def get_accuracy(selected_options):
  correct_count = 0
  total_count = len(selected_options)
  incorrect_questions = []

  for i, key in enumerate(list(answers_dict.keys())[:numbers_question]):
      predicted_answer = selected_options[i]
      correct_answer = answers_dict.get(key)

      if predicted_answer == correct_answer:
          correct_count += 1
      else:
          incorrect_questions.append(key)
          if (key not in error_question):
              error_question[key] = 1
          else:
              error_question[key] += 1


  accuracy = (correct_count / total_count) * 100

  return accuracy

In [ ]:
import time
import psutil
import os
from tqdm import tqdm
import numpy as np

# Get the current process
process = psutil.Process(os.getpid())

# Parameters
runs = 1  # Number of iterations (runs)
numbers_question = 100  # Number of questions per run
accu_arr = []  # Store accuracy for each run
time_per_question = []  # Store time for each question
total_memory_consumed = 0  # Track total memory consumption across all questions
total_questions = runs * numbers_question  # Total number of questions processed

# Simulate running the model
for i in tqdm(range(runs)):  # Loop over the number of runs
    returned_result = get_answer()  # Simulate getting answers from the model/API

    # Iterate over each question in the current run
    for j in range(numbers_question):
        # Record start time and memory usage
        start_time = time.time()
        start_memory = process.memory_info().rss / (1024 * 1024)  # Memory in MB

        # Simulate calculating accuracy for the current question
        accuracy = get_accuracy(returned_result)

        # Record end time and memory usage
        end_time = time.time()
        end_memory = process.memory_info().rss / (1024 * 1024)  # Memory in MB

        # Append time for the current question
        time_per_question.append(end_time - start_time)

        # Accumulate memory consumption
        total_memory_consumed += (end_memory - start_memory)

    # Append the overall accuracy for the current run
    accu_arr.append(accuracy)

# Calculate average time per question
average_time_per_question = np.mean(time_per_question)

# Calculate average memory usage across all questions (rounded to 3 decimal places)
average_memory_usage = round(total_memory_consumed / total_questions, 3)

# Print results
print("-------------------")
print("Accuracy per run is: ")
print(accu_arr)  # Accuracy for each run
print("Variance is: " + str(np.var(accu_arr)))  # Variance of accuracy across runs
print("-------------------")
print(f"Average time per question: {average_time_per_question:.20f} seconds")  # Average time
print(f"Average memory usage: {average_memory_usage:.50f} MB")  # Average memory usage


  0%|          | 0/1 [00:00<?, ?it/s]


APIStatusError: Error code: 402 - {'error': {'message': 'Insufficient Balance', 'type': 'unknown_error', 'param': None, 'code': 'invalid_request_error'}}